# Porting Yao.jl with QuantumInformation.jl
### GiggleLiu

# overview

 [`Yao`](https://github.com/QuantumBFS/Yao.jl) is a powerful tool for quantum circuit based simulation, but it does not support many density matrix related operations. This is why we need to port `Yao.jl` with [`QuantumInformation (QI)`](https://github.com/QuantumBFS/QuantumInformation.jl) sometimes (e.g. for computing entanglement entropy).
 
* `Yao.jl` Documentation: https://quantumbfs.github.io/Yao.jl/latest/ (paper is comming out)
* `QuantumInformation.jl` paper: https://arxiv.org/abs/1806.11464
     
### `Yao` provides
* high performance quantum circuit based simulation
    * parameter management
    * gradients
    * batched regiser
* automatic differentiation
* operator matrix representation and arithmatics
* [quantum algorithms](https://github.com/QuantumBFS/QuAlgorithmZoo.jl)
* [GPU support](https://github.com/QuantumBFS/CuYao.jl)
* basic density matrix simulation
* basic symbolic simulation
* qudits

### `QI` provides

* Compute entropy from density matrices
* Quantum channels, four types of channel representations
    * Kraus Operator
    * Super operator
    * Dynamic matrices
    * Stinespring representation
* Compute norm, distance and distingushability between "states" (density matrices)
    * Hilbert–Schmidt norm and distance
    * trace norm and *distance*
    * diamond norm
    * Bures distane and Bures angles
    * *fidelity* and superfidelity
    * KL-divergence
    * JS-distance
* Compute the amount of entanglement
     * negativity
     * positive partial trace
     * concurrence
* POVM measurements

In [2]:
import Yao
using Yao: arrayreg, ρ, mat, ConstGate, purify, exchange_sysenv, @bit_str, statevec
import QuantumInformation; const QI = QuantumInformation
using QuantumInformation: ket
using LinearAlgebra
using Test

┌ Info: Precompiling QuantumInformation [3c0b384b-479c-5684-b2ef-9d7a46dd931e]
└ @ Base loading.jl:1423
┌ Warning: please replace index ranges like `i:3` with `i in 1:3` or `i ∈ 1:3`
│   call.string = "@cast x[i, j] := ϕ[(j, i)]  j:cols"
└ @ TensorCast ~/.julia/packages/TensorCast/eabry/src/macro.jl:1040


Obtain reduced density matrices in Yao
-------------------------
The memory layout of `Yao` register and `QI` ket are similar, their basis are both [little endian](https://en.wikipedia.org/wiki/Endianness), despite they have different representation powers

* `Yao` support batch,
* `QI` is not limited to qubits.


`Yao` does not have much operations defined on density matrices, but purified states with environment,
On the other side, most operations in `QI` are defined on **(density) matrices**, they can be easily obtained in `Yao`.

In [17]:
# construct a product state, notice the indexing in `QI` starts from `1`
@test QI.ket(3, 1<<4) ≈ statevec(arrayreg(bit"0010"))

# join two registers, notice little endian convension is used here.
reg = join(arrayreg(bit"10"), arrayreg(bit"11"))
v = QI.:⊗(QI.ket(0b10+1,1<<2), QI.ket(0b11+1,1<<2))
@test statevec(reg) ≈ v

Test Passed

In [3]:
# e.g. obtain a reduced denstiy matrix for subsystem 1,2,3,4
reg = Yao.rand_state(10)
rho = Yao.density_matrix(reg, 1:4) # make qubits 1-4 active
rho.state

16×16 Matrix{ComplexF64}:
    0.0613256+0.0im          …    -0.0014683+0.00421717im
   0.00114577+0.00251627im      -0.000582502+0.00320995im
  -0.00745463-0.00584969im      -0.000252047-0.000884078im
  -0.00701895+0.00197244im        0.00718725+0.00547219im
   0.00391836+0.00974869im        -0.0022324-0.00455249im
   0.00526156+0.00459629im   …   -0.00292748-0.000824591im
 -0.000327233-0.00981608im       -0.00208897-0.00124306im
 -0.000401732+0.00678579im       -0.00731678-0.0106494im
    6.8507e-5-0.00321467im       -0.00483906-0.00658371im
   0.00485445-0.00129369im        -0.0030162+0.00357678im
  0.000588632-0.000120666im  …    0.00223251+0.000401994im
  -0.00578016+0.0117884im         0.00258094+0.00177805im
    0.0047982+0.00077583im       -0.00604365+0.00209272im
  -0.00187469+0.00304266im        -0.0010979+0.00707576im
  -0.00133934+0.00576856im        -0.0066598-0.00013793im
   -0.0014683-0.00421717im   …     0.0576982+0.0im

One can also convert a density matrix to a a quantum state through **purification**

In [5]:
# e.g. purify a state and recover it
reg = Yao.rand_state(6) |> Yao.focus!(1:4)
reg_p = purify(reg |> ρ; num_env=2)
@test Yao.fidelity(reg, reg_p)[] ≈ 1

Test Passed
  Expression: (Yao.fidelity(reg, reg_p))[] ≈ 1
   Evaluated: 1.0000000000000002 ≈ 1

entanglement & state distance
----------------


In [8]:
reg1 = Yao.rand_state(10)
freg1 = Yao.focus!(reg1, 1:4)
reg2 = Yao.rand_state(6)
freg2 = Yao.focus!(reg2, 1:4)
dm1, dm2 = freg1 |> Yao.density_matrix, freg2 |> Yao.density_matrix

# trace distance between two registers (different by a factor 2)
@test Yao.tracedist(freg1, freg2)[]/2 ≈ QI.trace_distance(dm1.state, dm2.state)

Test Passed
  Expression: (Yao.tracedist(freg1, freg2))[] / 2 ≈ QI.trace_distance(dm1.state, dm2.state)
   Evaluated: 0.7916813951218918 ≈ 0.7916813951218918

In [10]:
# get the entanglement entropy between system and env
@show QI.vonneumann_entropy(dm1.state)
@show QI.vonneumann_entropy(dm2.state)

QI.vonneumann_entropy(dm1.state) = 2.632001104147413
QI.vonneumann_entropy(dm2.state) = 1.2933417177660989


1.2933417177660989

In [11]:
# KL-divergence (or relative entropy)
QI.kl_divergence(dm2.state, dm1.state)

1.727464949101496

Note: you can defined many distances and entropies in a similar way, we don't enumerate it.

Quantum Operations/Quantum Gates
------------------------

A quantum gate in `Yao` is equivalent to a unitary channel in `QI`, matrix representations of blocks in `Yao` can be used to construct channels.

In [23]:
# applying a rotation gate
b1 = Yao.put(2,2=>Yao.Rx(0.3π))
c1 = QI.UnitaryChannel(mat(b1))
b2 = Yao.put(2,2=>Yao.Ry(0.3π))
c2 = QI.UnitaryChannel(mat(b2))

reg = Yao.rand_state(2)
@test copy(reg) |> b1 |> Yao.density_matrix |> Yao.state ≈ c1(reg |> Yao.density_matrix |> Yao.state)
@test copy(reg) |> Yao.chain(b1,b2) |> Yao.density_matrix |> Yao.state ≈ (c2∘c1)(reg |> Yao.density_matrix |> Yao.state)

Test Passed
  Expression: ((copy(reg) |> Yao.chain(b1, b2)) |> Yao.density_matrix) |> Yao.state ≈ (c2 ∘ c1)((reg |> Yao.density_matrix) |> Yao.state)
   Evaluated: ComplexF64[0.007685889275322289 + 0.0im -0.009977021258596053 - 0.009855156379047764im 0.050425125602665107 - 0.04974113723108121im -0.036121303372492815 + 0.03329457521138159im; -0.009977021258596053 + 0.009855156379047764im 0.02558780817743653 + 0.0im -0.0016765612617604625 + 0.12922588997185136im 0.004197271805229389 - 0.08953573924625202im; 0.050425125602665107 + 0.04974113723108121im -0.0016765612617604625 - 0.12922588997185136im 0.6527382642882836 + 0.0im -0.452456595399494 - 0.015330896253438902im; -0.036121303372492815 - 0.03329457521138159im 0.004197271805229389 + 0.08953573924625202im -0.452456595399494 + 0.015330896253438902im 0.31398803825895755 + 0.0im] ≈ ComplexF64[0.007685889275322329 + 3.469446951953614e-18im -0.009977021258596086 - 0.009855156379047755im 0.05042512560266519 - 0.049741137231081214im -0.036121

For more general non-unitary operations, we need the Kraus operator.

In [37]:
# construct a Kraus Operator
ko = QI.KrausOperators([
            sqrt(0.3)*Matrix(Yao.put(2, 1=>ConstGate.X)),
            sqrt(0.4)*Matrix(Yao.put(2, 2=>ConstGate.Y)), 
            sqrt(0.3)*Matrix(Yao.put(2, 2=>ConstGate.Z))
        ])

QuantumInformation.KrausOperators{Matrix{ComplexF64}}
    dimensions: (4, 4)
    ComplexF64[0.0 + 0.0im 0.5477225575051661 + 0.0im 0.0 + 0.0im 0.0 + 0.0im; 0.5477225575051661 + 0.0im 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im 0.5477225575051661 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im 0.5477225575051661 + 0.0im 0.0 + 0.0im]
    ComplexF64[0.0 + 0.0im 0.0 + 0.0im 0.0 - 0.6324555320336759im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im 0.0 - 0.6324555320336759im; 0.0 + 0.6324555320336759im 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.6324555320336759im 0.0 + 0.0im 0.0 + 0.0im]
    ComplexF64[0.5477225575051661 + 0.0im 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.5477225575051661 + 0.0im 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im -0.5477225575051661 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im -0.5477225575051661 + 0.0im]

It is equivalent to Yao's unitary channel defined as

In [38]:
uc = Yao.UnitaryChannel([Yao.put(2, 1=>ConstGate.X), Yao.put(2,2=>ConstGate.Y), Yao.put(2,2=>ConstGate.Z)],
                        [0.3, 0.4, 0.3])

nqubits: 2
unitary_channel
├─ [0.3] put on (1)
│  └─ X
├─ [0.4] put on (2)
│  └─ Y
└─ [0.3] put on (2)
   └─ Z


In [39]:
dm = Yao.density_matrix(Yao.rand_state(4), (1,2))
@test Yao.apply(dm, uc).state ≈ ko(dm.state)

(w, o) = (0.4, nqubits: 2
put on (2)
└─ Y)


Test Passed
  Expression: (Yao.apply(dm, uc)).state ≈ ko(dm.state)
   Evaluated: ComplexF64[0.25953844977919793 + 0.0im 0.022064060477886942 - 0.055158213086358925im -0.027858223441154083 + 0.03880795039681731im 0.044931623972655925 - 0.04778982053282649im; 0.022064060477886942 + 0.055158213086358925im 0.2139156647108312 + 0.0im 0.03152068993990473 + 0.00972015313089635im 0.06654396872766964 - 0.0006186343373243836im; -0.027858223441154083 - 0.03880795039681731im 0.03152068993990473 - 0.00972015313089635im 0.2580427607974538 + 0.0im 0.04317374592198403 + 0.04019871360769118im; 0.044931623972655925 + 0.04778982053282649im 0.06654396872766964 + 0.0006186343373243836im 0.04317374592198403 - 0.04019871360769118im 0.26850312471251697 + 0.0im] ≈ ComplexF64[0.2595384497791979 + 0.0im 0.02206406047788695 - 0.055158213086358925im -0.02785822344115408 + 0.03880795039681731im 0.04493162397265592 - 0.04778982053282648im; 0.02206406047788695 + 0.055158213086358925im 0.21391566471083118 + 0.0im 0.03